In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.lit



val conf=new SparkConf()
      .setMaster("local")//启动本地化计算
      .setAppName("testRdd")//设置本程序名称
val sc=new SparkContext(conf)
val spark = SparkSession.builder().appName("csv_app").getOrCreate()

conf = org.apache.spark.SparkConf@54c0a1d4
sc = org.apache.spark.SparkContext@10684ae9
spark = org.apache.spark.sql.SparkSession@2b531518


org.apache.spark.sql.SparkSession@2b531518

In [2]:
val csv1=spark.read.option("header", "true").option("multiLine", true).csv("../data/input.csv")//读取本地文件
val csv2=spark.read.option("header", "true").option("multiLine", true).csv("../data/attribute_mapping.csv")

csv1 = [StartDate: string, EndDate: string ... 3244 more fields]
csv2 = [Attribute: string, Electrolux: string ... 51 more fields]


[Attribute: string, Electrolux: string ... 51 more fields]

In [6]:
val col2 = csv2.columns
val brand = col2.slice(1, col2.length)
//col2.filter(!_.contains("Attribute"))
//col2.slice(col1.indexWhere(_=="xx"), col1.length)

brand = Array(Electrolux, AEG, Westinghouse, Zanussi, Simpson, Haier, Asko, Smeg, LG, Samsung, Fisher & Paykel, Bosch, Miele, Hisense, Kelvinator, Panasonic, Sharp, Toshiba, Hitachi, Daikin, Elba, Aqua, Teka, Midea, Beko, Rinnai, Fotile, Mitsubishi, Modena, Tec...


")


Array(Electrolux, AEG, Westinghouse, Zanussi, Simpson, Haier, Asko, Smeg, LG, Samsung, Fisher & Paykel, Bosch, Miele, Hisense, Kelvinator, Panasonic, Sharp, Toshiba, Hitachi, Daikin, Elba, Aqua, Teka, Midea, Beko, Rinnai, Fotile, Mitsubishi, Modena, Tec...

In [10]:
import org.apache.spark.sql.types._ //StructType, StructField
val schema = StructType(
  Seq(
    StructField("Attribute", StringType, true),
    StructField("Question", StringType, true),
    StructField("Brand", StringType, true)))

schema = StructType(StructField(Attribute,StringType,true), StructField(Question,StringType,true), StructField(Brand,StringType,true))


StructType(StructField(Attribute,StringType,true), StructField(Question,StringType,true), StructField(Brand,StringType,true))

In [63]:
import org.apache.spark.sql.Row
var emptydf = spark.createDataFrame(sc.emptyRDD[Row], schema)
emptydf.show

+---------+--------+-----+
|Attribute|Question|Brand|
+---------+--------+-----+
+---------+--------+-----+



emptydf = [Attribute: string, Question: string ... 1 more field]


[Attribute: string, Question: string ... 1 more field]

In [92]:
import org.apache.spark.sql.functions._
var kk = brand.map(name => col(name))
print(kk)

[Lorg.apache.spark.sql.Column;@2f7d301c

)


#  scala中利用“: _*” 可以实现函数的非定长参数 

In [125]:
var df = csv2.select(brand.slice(0,3).map(name => col(name)): _*)
df.show

+-------------------+-------------------+-------------------+
|         Electrolux|                AEG|       Westinghouse|
+-------------------+-------------------+-------------------+
|  sustainability_57|  sustainability_58|  sustainability_59|
|human-centricity_57|human-centricity_58|human-centricity_59|
|           style_57|           style_58|           style_59|
|     premiumness_57|     premiumness_58|     premiumness_59|
|      innovation_57|      innovation_58|      innovation_59|
|          status_57|          status_58|          status_59|
|       modernity_57|       modernity_58|       modernity_59|
|eco-friendliness_57|eco-friendliness_58|eco-friendliness_59|
|     reliability_57|     reliability_58|     reliability_59|
|         quality_57|         quality_58|         quality_59|
|   intuitiveness_57|   intuitiveness_58|   intuitiveness_59|
| value-for-money_57| value-for-money_58| value-for-money_59|
|    family-focus_57|    family-focus_58|    family-focus_59|
|       

df = [Electrolux: string, AEG: string ... 1 more field]


[Electrolux: string, AEG: string ... 1 more field]

In [126]:
import org.apache.spark.sql.functions._
//var df = csv2.select("Attribute").withColumn("Question", concat_ws(",", brand.map(name => col(name)): _*))
var df = csv2.select(col("Attribute"), concat_ws(",", brand.map(name => col(name)): _*).as("Question"))
df.show

+--------------------+--------------------+
|           Attribute|            Question|
+--------------------+--------------------+
|is a leader in su...|sustainability_57...|
|offers relevant s...|human-centricity_...|
|          is stylish|style_57,style_58...|
|          is premium|premiumness_57,pr...|
|is a leader in in...|innovation_57,inn...|
|is a brand with s...|status_57,status_...|
|has products with...|modernity_57,mode...|
|offers environmen...|eco-friendliness_...|
|offers durable an...|reliability_57,re...|
|offers high-quali...|quality_57,qualit...|
|      is easy to use|intuitiveness_57,...|
|offers good value...|value-for-money_5...|
|   is family focused|family-focus_57,f...|
|simplifies my hom...|simple_57,simple_...|
|   is a clever brand|clever_57,clever_...|
|  is a brand I trust|trust_57,trust_58...|
|   is a brand I love|love_57,love_58,l...|
+--------------------+--------------------+



df = [Attribute: string, Question: string]


[Attribute: string, Question: string]

In [117]:
df.select(col("Attribute"), explode(split(col("Question"),","))).withColumn("brand", csv2.$"Question")

+--------------------+-----------------+
|           Attribute|              col|
+--------------------+-----------------+
|is a leader in su...|sustainability_57|
|is a leader in su...|sustainability_58|
|is a leader in su...|sustainability_59|
|is a leader in su...|sustainability_95|
|is a leader in su...|sustainability_68|
|is a leader in su...|sustainability_60|
|is a leader in su...|sustainability_61|
|is a leader in su...|sustainability_62|
|is a leader in su...|sustainability_63|
|is a leader in su...|sustainability_64|
|is a leader in su...|sustainability_65|
|is a leader in su...|sustainability_66|
|is a leader in su...|sustainability_67|
|is a leader in su...|sustainability_70|
|is a leader in su...|sustainability_71|
|is a leader in su...|sustainability_72|
|is a leader in su...|sustainability_73|
|is a leader in su...|sustainability_74|
|is a leader in su...|sustainability_75|
|is a leader in su...|sustainability_76|
+--------------------+-----------------+
only showing top

In [61]:
var csv2DF = emptydf
for(i<-(0 until brand.length)){
    csv2DF = csv2DF.union(
    csv2.select("Attribute",brand(i))
        .toDF("Attribute", "Question")
        .withColumn("Brand", lit(brand(i)))
    )
}

csv2DF = [Attribute: string, Question: string ... 1 more field]


[Attribute: string, Question: string ... 1 more field]